In [11]:
import numpy as np
import pandas as pd
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import calendar
import datetime as dt
import datetime
import pickle
import azureml.train.automl
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from math import sqrt
from lifelines.utils import k_fold_cross_validation
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
import seaborn as sns
from sklearn.datasets.samples_generator import (make_blobs,
                                                make_circles,
                                                make_moons)
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score

%matplotlib inline
sns.set_context('notebook')
plt.style.use('fivethirtyeight')
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
#Reading the closed data and merging them together
q1=pd.read_csv('train_2019q1.csv')
q2=pd.read_csv('train_2019q2.csv')
q3=pd.read_csv('train_2019q3.csv')
q4=pd.read_csv('train_2018q4.csv')

df=q4.append([q1,q2,q3])
print(df.shape)

(3274451, 65)


In [3]:
dataframe = df.drop(columns=['invoice_nbr'])
dataframe.drop_duplicates(inplace=True)
print(dataframe.shape)

(1014863, 64)


In [4]:
dataframe['Paid'] = 1

dataframe['payment_due_date']=pd.to_datetime(dataframe['payment_due_date'])
dataframe['invoice_date']=pd.to_datetime(dataframe['invoice_date'])
dataframe['actual_close_date']=pd.to_datetime(dataframe['actual_close_date'])

#Calculating the Target variable
dataframe['DSO2'] = (dataframe['actual_close_date'] - dataframe['invoice_date']).dt.days

dataframe.head()

,customer_nbr,source_country_code,security_country_code,country_code,Invoice_Base_Amount_US,Commited_Days,Payment_Term,Invoice_Category,Count_Paid_Invoices,Count_Delayed_Invoice,Ratio_Delayed_Invoices,Paid_Invoices_amount_US,Dalayed_Paid_Invoices_Amount,Ratio_Late_Paid_Amount,Avg_Days_Late_Paid_Invoices_Closed,Count_Outstanding_Invoices,Count_Delayed_Outstanding_Invoice,Ratio_Outstanding_Invoices,Outstanding_Invoices_Amount,Dalayed_Outstanding_Invoices_Amount,Ratio_Late_Outstanding_Amount,Avg_Days_Late_Outstanding_Invoices_Open,DSO,Delay_Status,industry_name,invoice_date,aging_date,payment_due_date,Payment_Quarter,Count_Prepaid_Invoices,Ratio_Prepaid_Invoices,Invc_creation_month,Invc_commited_date_month,Actual_close_date_Month,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,Open_60_more,actual_close_date,ME_Due_Date,ME_Inv_Date,MME_Due_Date,MME_Inv_Date,Invoice_OKS,TM_Invoice,OM_Invoice,HSR_SCA_Invoice,SCA_Invoice,SAAS_OKS_Invoice,HSR_TM_Invoice,HSR_Invoice_OKS,HSR_OM_Invoice,Other_Invoice_Type_Name,WCS,GSH,GSR,GSF,ISF,SIT,ISR,NON,Other_functional_group_short_name,Paid,DSO2
0,2144793,0,US,US,272.440,30,30-Jan,0,13925,11642,0.836,20663334.670,17122020.160,0.829,35,13,13,1,104516.180,104516.180,1.000,19002360.670,0,0,Retail ...,2018-09-01,9/1/2018,2018-10-01,2018Q4,1293,0.093,9,10,10,2283,8120,1990,263,1269,13,2018-10-01,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,30
1,7430837,0,US,US,740.030,30,30-Jan,0,267755,249084,0.930,212355668.900,193551125.600,0.911,38,24440,24440,1,16265111.750,16265111.750,1.000,19000442.550,5,1,Hospitality ...,2018-10-23,10/23/2018,2018-11-22,2018Q4,18340,0.069,10,11,11,18671,178325,50715,6658,13386,24440,2018-11-27,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,35
2,36154,0,US,US,11453.800,30,30-Jan,0,1195,493,0.413,10529796.280,4716812.430,0.448,76,14,14,1,140509.640,140509.640,1.000,19000316.780,7,1,Financial ...,2018-10-26,10/26/2018,2018-11-25,2018Q4,695,0.582,10,11,12,702,331,12,45,105,14,2018-12-02,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,37
3,28291,0,US,US,432.020,30,30-Jan,0,45887,39692,0.865,55800919.340,37284690.510,0.668,117,651,651,1,239051.310,239051.310,1.000,19000793.450,35,2,Retail ...,2018-10-27,10/27/2018,2018-11-26,2018Q4,4691,0.102,10,11,12,6195,9419,10235,3436,16602,651,2018-12-31,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,65
4,9304719,0,US,US,130.350,30,30-Jan,0,16483,14417,0.875,3374119.850,2997863.140,0.888,30,112,112,1,20641.770,20641.770,1.000,19000463.410,-10,0,Retail ...,2018-09-21,9/21/2018,2018-10-21,2018Q4,2066,0.125,9,10,10,2066,8881,3443,1383,710,112,2018-10-11,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,20


In [5]:
dataframe2 = dataframe.loc[(dataframe['DSO2']>=0) & (dataframe['invoice_date'] >= '2018-01-01')]
print(dataframe2.shape)

(979866, 66)


In [6]:
#Reading in the open invoices

open_invoices=pd.read_csv('t2p_testdata_oct2019.csv')
open_invoices['payment_due_date']=pd.to_datetime(open_invoices['payment_due_date'])
open_invoices['invoice_date']=pd.to_datetime(open_invoices['invoice_date'], errors='coerce')
open_invoices['actual_close_date']=pd.to_datetime(open_invoices['actual_close_date'], errors='coerce')

open_invoices = open_invoices.loc[(open_invoices['invoice_date'] >= '2018-01-01') & (open_invoices['invoice_date'] < '2019-10-01')]

open_invoices['Paid'] = 0
open_invoices.drop(columns=['Unnamed: 65'], inplace=True)
open_invoices = open_invoices.loc[open_invoices['actual_close_date'].isna()]
print(open_invoices.shape)

(689616, 66)


In [7]:
open_invoices['Study_End_Date'] = '2019-10-01'
open_invoices['Study_End_Date'] = pd.to_datetime(open_invoices['Study_End_Date'])

#Calculating the Target variable
open_invoices['DSO2'] = (open_invoices['Study_End_Date'] - open_invoices['invoice_date']).dt.days
open_invoices.drop(columns=['Study_End_Date'], inplace=True)
print(open_invoices.shape)

(689616, 67)


In [ ]:
# open_invoices2 = open_invoices.drop(columns=['invoice_nbr'])
# open_invoices2.drop_duplicates(inplace=True)
# print(open_invoices2.shape)

In [8]:
#Appending the closed and open invoices to create the right censored dataset

total = dataframe2.append([open_invoices])
print(total.shape)

(1669482, 68)


In [9]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()

    return pd.Series((month_num, day_of_month, day_of_week))

In [10]:
total[["month_num_due", "day_of_month_due","day_of_week_due"]] = total[["payment_due_date"]].apply(build_time_features, axis=1)
total[["month_num_created", "day_of_month_created","day_of_week_created"]] = total[["invoice_date"]].apply(build_time_features, axis=1)

#Featurization of Aging Date
total['aging_date']=pd.to_datetime(total['aging_date'])
total[["month_num_age", "day_of_month_age","day_of_week_age"]] = total[["aging_date"]].apply(build_time_features, axis=1)

###### Only for North America

In [11]:
total2 = total.loc[(total['security_country_code'] == 'US') | (total['security_country_code'] == 'CA')]

na=pd.read_csv('North_America_Customers.csv')
na2=na[['customer_nbr', 'Flag']]
na2.rename(columns={'Flag':'Customer_Segment'}, inplace=True)

total3=total2.merge(na2,how='left', on='customer_nbr')

print(total2.shape)
print(total3.shape)

(1324359, 77)
(1324359, 78)


In [44]:
total3.to_csv('Entire_Dataset_Survival.csv', index=False)

In [28]:
final = total3[['security_country_code','Invoice_Base_Amount_US', 'Commited_Days', 'Invoice_Category', 'Count_Paid_Invoices', 'Count_Delayed_Invoice', 'Ratio_Delayed_Invoices', 'Paid_Invoices_amount_US', 'Dalayed_Paid_Invoices_Amount', 'Ratio_Late_Paid_Amount', 'Avg_Days_Late_Paid_Invoices_Closed', 'Count_Outstanding_Invoices', 'Count_Delayed_Outstanding_Invoice', 'Ratio_Outstanding_Invoices','Dalayed_Outstanding_Invoices_Amount', 'Ratio_Late_Outstanding_Amount', 'Avg_Days_Late_Outstanding_Invoices_Open',
       'industry_name','Payment_Quarter', 'Count_Prepaid_Invoices', 'Ratio_Prepaid_Invoices','closed_bucket_0', 'closed_bucket_1', 'closed_bucket_2', 'closed_bucket_3', 'closed_bucket_4', 'Open_60_more',
       'ME_Due_Date', 'ME_Inv_Date', 'MME_Due_Date', 'MME_Inv_Date', 'Invoice_OKS', 'TM_Invoice', 'OM_Invoice', 'HSR_SCA_Invoice', 'SCA_Invoice', 'SAAS_OKS_Invoice', 'HSR_TM_Invoice', 'HSR_Invoice_OKS', 'HSR_OM_Invoice', 'Other_Invoice_Type_Name', 'WCS', 'GSH', 'GSR', 'GSF', 'ISF', 'SIT', 'ISR', 'NON', 'Other_functional_group_short_name', 'month_num_due', 'day_of_month_due',
        'day_of_week_due', 'month_num_created', 'day_of_month_created', 'day_of_week_created','month_num_age', 'day_of_month_age','day_of_week_age', 'DSO2', 'Paid','Customer_Segment']]
print(final.shape)

(1324359, 62)


In [30]:
final2 =final.drop_duplicates()
print(final2.shape)

(929208, 62)


In [31]:
final3 = final2.dropna()
print(final3.shape)

(914123, 62)


In [32]:
final3.to_csv('Train_Survival.csv', index=False)

In [33]:
train = pd.read_csv('Train_Survival.csv')